In [743]:
%matplotlib notebook
import IPython.display
from utils_demo import *
from sys import platform
import sys
from PIL import Image
import matplotlib.pyplot as plt
import json
import rdflib
import glob
import os
import re
import copy
import time

sys.path.append('../simulation')
sys.path.append('../dataset_utils/')

import numpy as np
import random
import cv2
import add_preconds
import augmentation_utils
import evolving_graph.check_programs as check_programs
import evolving_graph.utils as utils

from unity_simulator.comm_unity import UnityCommunication

In [744]:
comm = UnityCommunication()

Select scene number. 
Please see https://github.com/xavierpuigf/virtualhome/tree/master/simulation#environment .

In [745]:
def generate_list_of_steps(file_path):
    file = open(file_path, "r", encoding="utf-8")
    i = 0
    list_of_steps = []
    program_name = ""
    description = ""
    char= "<char0>"
    while True:
        line = file.readline()
        if line:
            line = line.replace("\n","")
            if i==0:
                program_name = line
            elif i==1:
                description = line
            elif line.startswith("["):
                list_of_steps.append(line)
            else:
                pass
            i+=1
        else:
            break
    return program_name, description, list_of_steps

In [746]:
scene = 1
data_path = "../dataset/experiment_programs_202203/scene" + str(scene) + "/*.txt"
program_list = []
for file_path in glob.glob(data_path):
    file_name = file_path.replace("../dataset/experiment_programs_202203/scene" + str(scene) + "/", "")
#     txt_file_name = file_name.split("/")[1]
    
    program_name, description, list_of_steps = generate_list_of_steps(file_path)
    program_list.append({
        "file_name":file_name,
        "name": program_name,
        "description": description,
        "list_of_steps": list_of_steps
    })

In [747]:
program_list

[{'file_name': 'Watch_tv.txt',
  'name': 'Watch tv',
  'description': 'Walk to the livingroom and then walk to the tv. Switch on the tv.',
  'list_of_steps': ['[WALK] <livingroom> (336)',
   '[WALK] <tv> (427)',
   '[SWITCHON] <tv> (427)',
   '[WALK] <sofa> (369)',
   '[TURNTO] <tv> (427)',
   '[SIT] <sofa> (369)']}]

In [748]:
def update_bbox(pre_graph, current_graph):
    try:
        new_graph= copy.deepcopy(current_graph)
        
        for pre_node in pre_graph["nodes"]:
            for new_node in new_graph["nodes"]:
                if new_node["id"] == pre_node["id"]:
                    new_node["bounding_box"] = pre_node["bounding_box"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [749]:
def update_states(pre_graph, graph_state, object_id_map):
    try:
        new_graph= copy.deepcopy(pre_graph)
        
        skip = [object_id_map[x] for x in object_id_map]
        for new_node in new_graph["nodes"]:
            new_node_id = new_node["id"]
            if new_node_id in skip:
                continue
            temp_node_id = copy.deepcopy(new_node_id)
            # scriptで使用したobject_idとgraph_stateで反映されたidが異なる場合
            if str(new_node_id) in object_id_map:
                temp_node_id = object_id_map[str(new_node_id)]
            for graph_state_node in graph_state["nodes"]:
                if temp_node_id == graph_state_node["id"]:
                    new_node["states"] = graph_state_node["states"]
                    new_node["properties"] = graph_state_node["states"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [750]:
def create_object_id_map(original_script, executed_program, g):
    try:
        object_id_map = {}
        object_name_map = {}
        for original_block_str, executed_block_str in zip(original_script, executed_program):
            original_action, original_obj_names_corr, original_inst_nums_corr = augmentation_utils.parseStrBlock(original_block_str)
            executed_action, executed_obj_names_corr, executed_inst_nums_corr = augmentation_utils.parseStrBlock(executed_block_str)
            
            #object_idのmapを作る
            for original_id, executed_id in zip(original_inst_nums_corr, executed_inst_nums_corr):
                if original_id not in object_id_map:
                    if original_id != executed_id:
                        object_id_map[original_id] = int(executed_id)
            
            #object_nameのmap作る
            for original_obj_name, original_id in zip(original_obj_names_corr, original_inst_nums_corr):
                g_objects = [x for x in g["nodes"] if x["id"] == original_id]
                if len(g_objects) != 0:
                    g_obj = g_objects[0]
                    if original_obj_name != g_obj["class_name"]:
                        object_name_map[original_obj_name] = g_obj["class_name"]
    except Exception as e:
        print(e)
    return object_id_map, object_name_map

In [751]:
def delete2000(graph_state_list):
    new_graph_state_list = []
    for graph_state in graph_state_list:
        new_nodes = [x for x in graph_state["nodes"] if x["id"] < 2000]
        new_edges = [x for x in graph_state["edges"] if x["from_id"] < 2000 and x["to_id"] < 2000]
        new_graph_state_list.append({"nodes": new_nodes, "edges": new_edges})
    return new_graph_state_list

In [752]:
def loadFoodInFridge(g):
    freezer = find_nodes(g, class_name="fridge")[0]
    freezer["states"] = ["OPEN"]
    add_node(g, {'class_name': 'pie', 
                   'category': 'Food', 
                   'id': 1000, 
                   'properties': [], 
                   'states': []})
    add_edge(g, 1000, 'INSIDE', freezer['id'])
    comm.expand_scene(g)
    success, g = comm.environment_graph()
    freezer = find_nodes(g, class_name="fridge")[0]
    freezer["states"] = ["CLOSED"]
    comm.expand_scene(g)
    success, g = comm.environment_graph()
    return success, g

In [753]:
def loadObjectInCabinet(g):
    try:
        cabinets = find_nodes(g, class_name="cabinet")
        cabinet = [x for x in cabinets if x["id"] == 416][0]

        cabinet["states"] = ["OPEN"]
        add_node(g, {'class_name': 'pie', 
                   'category': 'Food', 
                   'id': 1000, 
                   'properties': [], 
                   'states': []})
        add_edge(g, 1000, 'INSIDE', cabinet['id'])
        comm.expand_scene(g)
        success, g = comm.environment_graph()
        cabinets = find_nodes(g, class_name="cabinet")
        cabinet = [x for x in cabinets if x["id"] == 416][0]
        cabinet["states"] = ["CLOSED"]
        comm.expand_scene(g)
        success, g = comm.environment_graph()
    except Exception as e:
        print(e)
    return success, g

In [754]:
def switchOff(g, class_names):
    for class_name in class_names:
        objects = find_nodes(g, class_name=class_name)
        for obj in objects:
            states = [x for x in obj["states"] if x != "ON"]
            if "OFF" not in states:
                states.append("OFF")
            obj["states"] = states
#             for node in g["nodes"]:
#                 if node["id"] == obj["id"]:
#                     node = obj
        return g

In [755]:
def export(activity_name, graph_state_list, time_list, script, object_name_map):
    os.mkdir("graph_state_list_with_bbox_png/experiment_programs_202203_scene" + str(scene) + "/" + activity_name)
    state_cnt = 0
    for graph_state in graph_state_list:
        state_cnt += 1
        file_path = "graph_state_list_with_bbox_png/experiment_programs_202203_scene" + str(scene) + "/"  + activity_name + "/activityList-graph-state-" + '{0:03d}'.format(state_cnt) + ".json"
        with open(file_path, 'w') as outfile:
            json.dump(graph_state, outfile)

    with open("graph_state_list_with_bbox_png/experiment_programs_202203_scene" + str(scene) + "/" + activity_name + "/activityList-program.txt", 'w') as f:
        for s in script:
            #ここでscriptのobject_nameを置換する処理書く
            for x in object_name_map:
                if x in s:
                    s = s.replace("<" + x  + ">", "<" + object_name_map[x] + ">")
            f.write("%s\n" % s)

    with open("graph_state_list_with_bbox_png/experiment_programs_202203_scene" + str(scene) + "/" + activity_name + "/program-description.txt", 'w') as f:
        f.write("%s\n" % activity["name"])
        f.write("%s\n" % activity["description"])
        
    time_list = [str(time) for time in time_list]
    duration = "\n".join(time_list)
    with open("graph_state_list_with_bbox_png/experiment_programs_202203_scene" + str(scene) + "/" + activity_name + "/duration.txt", 'w') as f:
        f.write(duration)

In [756]:
def check_script_type(script):
    find_solution = False
    fridge = False
    if "(1)" in script[0]:
        find_solution = True
    for x in script:
        if "[OPEN] <fridge>" in x:
            fridge = True
    return find_solution, fridge

In [760]:
#find_solution = false
for activity in program_list:
    activity_name = activity["name"]
    try:
        new_graph_state_list = []
        time_list = []
        comm.reset(scene-1)
#         comm.add_character('chars/Female4')
        comm.add_character()
        success, g = comm.environment_graph();
        print(activity["file_name"])
        script = activity["list_of_steps"]
        
        find_solution, fridge = check_script_type(script)
        script_for_check = []
        if find_solution:
            continue
        else:
            for x in script:
                script_for_check.append(re.sub("(\d+)","1",x))
            print("script for check")
            print(script_for_check)

        preconds = add_preconds.get_preconds_script(script).printCondsJSON()
        print(preconds)
        print("switch off")
        g = switchOff(g, ["computer", "tv", "faucet", "television"])
        
        g = check_programs.translate_graph_dict_nofile(g)
        info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
        message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
        # if delete randomly placced objects
        graph_state_list = delete2000(graph_state_list)
        print(message)
        executed_program = []
        for x in executed_script:
            executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())
        print(executed_program)

        if message != "Script is executable":
            continue
        
        new_script = []
        
        for x in script:
            new_script.append("<char0> " + x)
            
        i = 0

        object_id_map, object_name_map = create_object_id_map(new_script, executed_program, g)
        print("object_id_map")
        print(object_id_map)
    
        success, g = comm.environment_graph()
#         g = update_bbox(g, graph_state_list[i])
        g = update_states(g, graph_state_list[i], object_id_map)
        new_graph_state_list.append(copy.deepcopy(g))
        
        for instruction in new_script:
            i += 1
            start = time.time()
            success, message = comm.render_script(script=[instruction],
                                      #processing_time_limit=80,
                                      find_solution=False,
                                      image_width=480,
                                      image_height=360,  
                                      #skip_animation=False,
                                      recording=True,
    #                                               gen_vid=True,
                                      #save_scene_states=True,
                                      file_name_prefix=activity_name,
                                      output_folder="Output/experiment_programs_202203/scene" + str(scene),
                                      frame_rate=20
                                     )
            time_list.append(time.time() - start)
            success, g = comm.environment_graph()
            print(str(success) + ": " + str(instruction))
#             g = update_bbox(g, graph_state_list[i])
            g = update_states(g, graph_state_list[i], object_id_map)
            new_graph_state_list.append(copy.deepcopy(g))
            
        export(activity_name, new_graph_state_list, time_list, new_script, object_name_map)
#                 with open("Output/" + activity_name + activity_cnt + "/" + str(scene) + "/ftaa_" + activity_name + activity_cnt + ".txt")  as f:
#                     line = f.readline()
#                     line.split(" ")
    except Exception as e:
        print(e)

Watch_tv.txt
script for check
['[WALK] <livingroom> (1)', '[WALK] <tv> (1)', '[SWITCHON] <tv> (1)', '[WALK] <sofa> (1)', '[TURNTO] <tv> (1)', '[SIT] <sofa> (1)']
[{'atreach': [['tv', '427'], ['sofa', '369']]}, {'free': ['sofa', '369']}]
switch off
Script is executable
['<char0> [WALK] <home_office> (336)', '<char0> [WALK] <television> (265)', '<char0> [SWITCHON] <television> (265)', '<char0> [WALK] <couch> (1000)', '<char0> [TURNTO] <television> (265)', '<char0> [SIT] <couch> (1000)']
object_id_map
{'427': 265, '369': 1000}
True: <char0> [WALK] <livingroom> (336)
True: <char0> [WALK] <tv> (427)
True: <char0> [SWITCHON] <tv> (427)
True: <char0> [WALK] <sofa> (369)
True: <char0> [TURNTO] <tv> (427)
True: <char0> [SIT] <sofa> (369)


In [761]:
#complementing missing images
try:
    for activity in program_list:
        activity_name = activity["name"]
        img_path = "../simulation/Output/experiment_programs_202203/scene" + str(scene) + "/" + activity_name + "/0/*.png"
        i = 0
        for file_path in sorted(glob.glob(img_path)):
            file_name = file_path.replace("../simulation/Output/experiment_programs_202203/scene" + str(scene) + "/" + activity_name + "/0/", "")
            file_num = file_name.split("_")[1]
            if i != int(file_num):
                print(str(i) + " " + str(int(file_num)))
                img = PIL.Image.open(file_path)
                new_file_name = "../simulation/Output/experiment_programs_202203/scene" + str(scene) + "/" + activity_name + "/0/" + "Action_%04d_0_normal.png"
                img.save(new_file_name % i)
                print(file_name)
                print(file_name + " - 1")
                print("save")
                i += 1
            i += 1
except Exception as e:
    print(e)

213 214
Action_0214_0_normal.png
Action_0214_0_normal.png - 1
save
235 236
Action_0236_0_normal.png
Action_0236_0_normal.png - 1
save
295 296
Action_0296_0_normal.png
Action_0296_0_normal.png - 1
save
356 357
Action_0357_0_normal.png
Action_0357_0_normal.png - 1
save
361 362
Action_0362_0_normal.png
Action_0362_0_normal.png - 1
save


## Please quit the exec.2.2.4 and run the exec_mac.

In [94]:
#find_solution = True
for activity in program_list:
    activity_name = activity["name"]
    try:
        new_graph_state_list = []
        time_list = []
        comm.reset(scene-1)
        comm.add_character('chars/Female4')
        success, g = comm.environment_graph();
        print(activity["file_name"])
        script = activity["list_of_steps"]
        
        find_solution, fridge = check_script_type(script)
        script_for_check = []
        if not find_solution:
            continue
        else:
            for x in script:
                script_for_check.append(re.sub("(\d+)","1",x))
            print("script for check")
            print(script_for_check)

        preconds = add_preconds.get_preconds_script(script).printCondsJSON()
        print(preconds)
        g = check_programs.translate_graph_dict_nofile(g)
        info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
        message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
        
        print(message)
        executed_program = []
        for x in executed_script:
            executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())

        if message != "Script is executable":
            continue
        
        print(executed_program)
        graph_state_list = delete2000(graph_state_list)
        comm.expand_scene(graph_state_list[0])
        success, g = comm.environment_graph()
        # if delete randomly placced objects
        
        i = 0
        
        if fridge:
            print("fridge")
            loadFoodInFridge(g)
        
        
        g = update_bbox(g, graph_state_list[i])
        new_graph_state_list.append(copy.deepcopy(g))

        script_find_solution = []
        for x in script:
            script_find_solution.append("<char0> " + x)
            
        script_for_rendering = None
        find_solution_flag = True
        if fridge:
            script_for_rendering = script_find_solution
        else:
            script_for_rendering = executed_program
            find_solution_flag = False
        
        for instruction in script_for_rendering:
            i += 1
            start = time.time()
            success, message = comm.render_script(script=[instruction],
                                      #processing_time_limit=80,
                                      find_solution=find_solution_flag,
                                      image_width=480,
                                      image_height=360,  
                                      #skip_animation=False,
                                      recording=True,
    #                                               gen_vid=True,
                                      #save_scene_states=True,
                                      file_name_prefix=activity_name,
                                      output_folder="Output/experiment_programs_202203/scene" + str(scene),
                                      frame_rate=20
                                     )
            time_list.append(time.time() - start)
            success, g = comm.environment_graph()
            print(str(success) + ": " + str(instruction))
            g = update_bbox(g, graph_state_list[i])
            new_graph_state_list.append(copy.deepcopy(g))
            
        if os.path.exists("graph_state_list_with_bbox_png/experiment_programs_202203_scene" + str(scene) + "/" + activity_name) == False:
            os.mkdir("graph_state_list_with_bbox_png/experiment_programs_202203_scene" + str(scene) + "/" + activity_name)
#         export(activity_name, new_graph_state_list, str(activity_cnt), time_list)
#                 with open("Output/" + activity_name + activity_cnt + "/" + str(scene) + "/ftaa_" + activity_name + activity_cnt + ".txt")  as f:
#                     line = f.readline()
#                     line.split(" ")
    except Exception as e:
        print(e)

Go_to_toilet.txt
Read_book.txt
Prepare_breakfast.txt
Wash_clothes.txt
Browse_Internet.txt
script for check
['[WALK] <home_office> (1)', '[WALK] <computer> (1)', '[SWITCHON] <computer> (1)', '[WALK] <chair> (1)', '[SIT] <chair> (1)']
[{'location': [['computer', '1'], ['home_office', '1']]}, {'location': [['chair', '1'], ['home_office', '1']]}, {'free': ['chair', '1']}]
Script is executable
['<char0> [WALK] <home_office> (336)', '<char0> [WALK] <computer> (434)', '<char0> [SWITCHON] <computer> (434)', '<char0> [WALK] <chair> (373)', '<char0> [SIT] <chair> (373)']
True: <char0> [WALK] <home_office> (336)
True: <char0> [WALK] <computer> (434)
True: <char0> [SWITCHON] <computer> (434)
True: <char0> [WALK] <chair> (373)
True: <char0> [SIT] <chair> (373)
Drink.txt
Take_a_nap.txt
Relax_on_sofa.txt
script for check
['[WALK] <home_office> (1)', '[WALK] <couch> (1)', '[WALK] <pillow> (1)', '[GRAB] <pillow> (1)', '[WALK] <couch> (1)', '[SIT] <couch> (1)']
[{'location': [['couch', '1'], ['home_offi

In [34]:
script = [
    "[WALK] <kitchen> (1)",
    "[WALK] <fridge> (1)",
    "[OPEN] <fridge> (1)",
    "[WALK] <pie> (1)",
    "[GRAB] <pie> (1)",
    "[CLOSE] <fridge> (1)",
    "[WALK] <microwave> (1)",
    "[OPEN] <microwave> (1)",
    "[PUTIN] <pie> (1) <microwave> (1)",
    "[CLOSE] <microwave> (1)",
    "[SWITCHON] <microwave> (1)"
]

print('Starting scene...')


try:
    message = comm.reset(scene-1)
    comm.add_character("Chars/Female4")
    success, g = comm.environment_graph();
    
    
    preconds = add_preconds.get_preconds_script(script).printCondsJSON()
    print(preconds)
    g = check_programs.translate_graph_dict_nofile(g)
    info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
    message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
    graph_state_list = delete2000(graph_state_list)
    executed_program = []
    for x in executed_script:
        executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())
    if message == "Script is executable":
        comm.expand_scene(graph_state_list[0])
        success, g = comm.environment_graph()
        
        # put the food in the freezer
        success, g = loadFoodInFridge(g)


        print(executed_program)
        print('Generating video...')
        new_script = []
        for x in script:
            new_script.append("<char0> " + x)
        success, message = comm.render_script(script=new_script, #cook some food　では　new_script
                                              #processing_time_limit=80,
                                              find_solution=True, # cook some food では True
                                              image_width=480,
                                              image_height=360,  
                                              #skip_animation=False,
                                              recording=True,
#                                               gen_vid=True,
                                              #save_scene_states=True,
                                              file_name_prefix="Cook_some_food",
                                              output_folder="Output/experiment_programs_202203/scene1",
                                              frame_rate=20
                                             )
        print(message)
#         imgs_final = get_scene_cameras(comm, [40])
    else:
        print(message)
except Exception as e:
    print(e)

Starting scene...
[{'free': ['chair', '1']}]
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [ ]:
freezer = find_nodes(g, class_name="fridge")[0]
freezer["states"] = ["CLOSED"]
comm.expand_scene(g)
imgs_final = get_scene_cameras(comm, [40])

In [ ]:
message = comm.reset(scene-1)
success, g = comm.environment_graph();
freezer = find_nodes(g, class_name="fridge")[0]
freezer["states"] = ["OPEN"]
comm.expand_scene(g)
imgs_final = get_scene_cameras(comm, [40])

In [ ]:
#電子レンジの上にグラス
message = comm.reset(scene-1)
success, g = comm.environment_graph();
sink = find_nodes(g, class_name="rug")[4]
add_node(g, {'class_name': 'pillow', 
               'category': 'placable_objects', 
               'id': 1000, 
               'properties': [], 
               'states': []})
add_edge(g, 1000, 'ON', sink['id'])
add_edge(g, 1000, 'CLOSE', 369)
comm.expand_scene(g)
success, g = comm.environment_graph()
imgs_final = get_scene_cameras(comm, [59])

In [210]:
message = comm.reset(0)
success, g = comm.environment_graph();
suceess, message = comm.get_visible_objects(57)
imgs_final = get_scene_cameras(comm, [57])
message

{'207': 'kitchen',
 '336': 'livingroom',
 '369': 'sofa',
 '372': 'coffeetable',
 '418': 'rug',
 '422': 'pillow',
 '423': 'pillow',
 '438': 'apple',
 '439': 'apple',
 '440': 'bananas',
 '442': 'peach',
 '443': 'peach',
 '444': 'plum',
 '445': 'plum',
 '446': 'dishbowl',
 '450': 'cellphone'}

In [338]:
message = comm.reset(scene-1)
comm.add_character()
success, g = comm.environment_graph();
script = [
    "[WALK] <tv> (427)",
    "[SWITCHON] <tv> (427)",
    "[WALK] <sofa> (369)",
    "[TURNTO] <tv> (427)",
    "[SIT] <sofa> (369)",
    "[WATCH] <tv> (427)"
#     "[DRINK] <wineglass> (199)",
#     "[WALK] <kitchen> (207)",
#     "[WALK] <sink> (247)",
#     "[PUTBACK] <wineglass> (199)"
#     "[WALK] <floor> (82)",
#     "[PUTBACK] <pillow> (423) <floor> (82)"
#     "[GRAB] <hanger> (115)",
#     "[WALK] <floor> (79)", 
#     "[TURNTO] <wallpictureframe> (419)",
#     "[LOOKAT] <wallpictureframe> (419)"
#     "[WALK] <rug> (179)",
#     "[PUTBACK] <hanger> (115) <floor> (79)",
#     "[WALK] <livingroom> (336)",
#     "[WALK] <sofa> (369)",
#     "[SIT] <sofa> (369)"
#     "[WALK] <computer> (1)",
#     "[SWITCHON] <computer> (1)",
#     "[WALK] <chair> (1)",
#     "[SIT] <chair> (1)"
#     "[OPEN] <washing_machine> (73)"
#     "[PUTBACK] <paper> (303) <desk> (110)"
#     "[GRAB] <clock> (266)",
#     "[WALK] <livingroom> (336)",
#     "[WALK] <coffeetable> (372)",
#     "[PUTBACK] <clock> (266) <coffeetable> (372)"
#     "[PUTBACK] <mug> (196) <sink> (247)"
#     "[SIT] <chair> (109)"
#     "[STANDUP]",
#     "[TURNTO] <toilet> (46)",
#     "[TOUCH] <toilet> (46)",
#     "[WALK] <sink> (248)",
#     "[SWITCHON] <faucet> (51)",
#     "[SWITCHOFF] <faucet> (51)"
#     "[GRAB] <plate> (274)",
#     "[WALK] <kitchencounter> (238)",
#     "[PUTBACK] <plate> (274) <kitchencounter> (238)",
#     "[WALK] <breadslice> (310)",
#     "[GRAB] <breadslice> (310)",
#     "[GRAB] <breadslice> (311)",
#     "[WALK] <plate> (274)",
#     "[PUTBACK] <breadslice> (310) <plate> (274)",
#     "[PUTBACK] <breadslice> (311) <plate> (274)",
#     "[WALK] <salmon> (328)",
#     "[GRAB] <salmon> (328)",
#     "[WALK] <plate> (274)",
#     "[WALK] <coffeepot> (308)",
#     "[GRAB] <coffeepot> (308)",
#     "[WALK] <kitchentable> (231)",
#     "[PUTBACK] <coffeepot> (308) <kitchentable> (231)"
#     "[SWITCHON] <faucet> (249)",
#     "[WALK] <sink> (247)",
#     "[PUTBACK] <glass> (271) <sink> (247)",
#     "[SWITCHOFF] <faucet> (249)",
#     "[GRAB] <glass> (271)",
#     "[DRINK] <glass> (271)"
]
try:
    preconds = add_preconds.get_preconds_script(script).printCondsJSON()
    print(preconds)
    g = check_programs.translate_graph_dict_nofile(g)
    info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
    message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
    
#     comm.expand_scene(graph_state_list[0])
    success, g = comm.environment_graph()
    
    executed_program = []
    for x in executed_script:
        instruction = "<char0> " + re.sub("\[\d+\]","",x.__str__()).strip()
#         instruction = instruction.replace("cup", "waterglass")
        executed_program.append(instruction)
#     graph_state_list = delete2000(graph_state_list)
    print(message)
    print(executed_program)
    new_script = []
    for x in script:
        new_script.append("<char0> " + x)
    success, message = comm.render_script(script=new_script, #cook some food　では　new_script
                                                  #processing_time_limit=80,
                                                  find_solution=False, # cook some food では True
                                                  image_width=480,
                                                  image_height=360,  
                                                  #skip_animation=False,
                                                  recording=True,
    #                                               gen_vid=True,
                                                  #save_scene_states=True,
    #                                               file_name_prefix=activity_name,
                                                  output_folder="Output/test",
#                                                   camera_mode=[58],
                                                  frame_rate=20
                                                 )
except Exception as e:
    print(e)

[{'atreach': [['tv', '427'], ['sofa', '369']]}, {'free': ['sofa', '369']}]
Script is executable
['<char0> [WALK] <television> (427)', '<char0> [SWITCHON] <television> (427)', '<char0> [WALK] <couch> (369)', '<char0> [TURNTO] <television> (427)', '<char0> [SIT] <couch> (369)', '<char0> [WATCH] <television> (427)']


In [155]:
executed_program

['<char0> [WALK] <dresser> (1000)',
 '<char0> [GRAB] <hanger> (115)',
 '<char0> [WALK] <bedroom> (74)',
 '<char0> [PUTBACK] <hanger> (115) <bedroom> (74)']

In [ ]:
imgs_final = get_scene_cameras(comm, [42])

In [161]:
sink = find_nodes(g, class_name="floor")
sink

[{'id': 12,
  'category': 'Floor',
  'class_name': 'floor',
  'prefab_name': 'PRE_FLO_Tile_07',
  'obj_transform': {'position': [-6.385, -0.003, -0.527],
   'rotation': [0.0, 0.0, 0.0, 1.0],
   'scale': [1.0, 1.0, 1.0]},
  'bounding_box': {'center': [-7.635, -0.003, -0.52699995],
   'size': [2.5, 9.050127e-06, 2.5]},
  'properties': ['SURFACES'],
  'states': []},
 {'id': 13,
  'category': 'Floor',
  'class_name': 'floor',
  'prefab_name': 'PRE_FLO_Tile_07',
  'obj_transform': {'position': [-6.385, -0.003, 1.973],
   'rotation': [0.0, 0.0, 0.0, 1.0],
   'scale': [1.0, 1.0, 1.0]},
  'bounding_box': {'center': [-7.635, -0.003, 1.973],
   'size': [2.5, 9.050127e-06, 2.5]},
  'properties': ['SURFACES'],
  'states': []},
 {'id': 14,
  'category': 'Floor',
  'class_name': 'floor',
  'prefab_name': 'PRE_FLO_Tile_07',
  'obj_transform': {'position': [-3.88500023, -0.003, 1.973],
   'rotation': [0.0, 0.0, 0.0, 1.0],
   'scale': [1.0, 1.0, 1.0]},
  'bounding_box': {'center': [-5.135, -0.003, 1.97

In [ ]:
cabinets = find_nodes(g, class_name="cabinet")
cabinet = [x for x in cabinets if x["id"] == 416][0]

cabinet["states"] = ["OPEN"]
comm.expand_scene(g)
imgs_final = get_scene_cameras(comm, [61])

In [ ]:
objects = find_nodes(g, class_name="mousemat")
objects

In [ ]:
loadObjectInCabinet(g)

In [64]:
action

'WALK'

In [65]:
obj_names

['sink']

In [66]:
inst_nums

['247']